In [1]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from Preprocessing import load_images
from keras.utils import np_utils
from sklearn import metrics as sk
from sklearn.metrics import cohen_kappa_score
np.random.seed(1337)
%matplotlib inline

Using TensorFlow backend.


In [2]:
tk.Tk().withdraw()
path_Train=tkFileDialog.askdirectory()
print path_Train

/home/edward/project /Dataset/Training_images Segmented


In [3]:
X_Train=load_images(path_Train)

In [4]:
X_Train=np.array(X_Train)
X_Train.shape

(2000, 64, 64, 3)

In [5]:
X_Train=np.reshape(X_Train,[-1,np.prod(X_Train.shape[1:])])
X_Train.shape

(2000, 12288)

In [6]:
tk.Tk().withdraw()
path_Test=tkFileDialog.askdirectory()
print path_Test

/home/edward/project /Dataset/Testing_images Segmented


In [7]:
X_Test=load_images(path_Test)

In [8]:
X_Test=np.array(X_Test)
X_Test.shape

(600, 64, 64, 3)

In [9]:
X_Test=np.reshape(X_Test,[-1,np.prod(X_Test.shape[1:])])
X_Test.shape

(600, 12288)

In [10]:
tk.Tk().withdraw()
path_Val=tkFileDialog.askdirectory()
print path_Val

/home/edward/project /Dataset/Validation_images Segmented


In [11]:
X_Val=load_images(path_Val)
X_Val=np.array(X_Val)
X_Val.shape

(150, 64, 64, 3)

In [12]:
X_Val=np.reshape(X_Val,[-1,np.prod(X_Val.shape[1:])])
X_Val.shape

(150, 12288)

In [13]:
data_Train=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Test_v2_Part3_GroundTruth.csv")
data_Val=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Validation_Part3_GroundTruth.csv")

In [14]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]
data_Val=data_Val.iloc[0:150,1]
print data_Train.shape
print data_Test.shape
print data_Val.shape

(2000,)
(600,)
(150,)


In [15]:
y_Train=data_Train
y_test=data_Test
y_Val=data_Val

In [16]:
y_Train=np.array(y_Train)
y_Test=np.array(y_test)
y_Val=np.array(y_Val)

In [17]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)
y_Val = np_utils.to_categorical(data_Val)

In [18]:
learning_rate=0.005
training_epoch=15
batch_size=100
display_step=1
model_path = "/home/edward/Trained models/model_sig.ckpt"
n_hidden_1=256
n_hidden_2=256
n_input=64*64*3
n_classes=2

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])

#### Model with Sigmoid Function

In [41]:
def multiplayer_perceptron(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [42]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron(x,weights,biases)

# define loss and optimizer
predictions=tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)
cost=tf.reduce_mean(predictions)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [43]:
# save model with sigmoid Activation function
#saver_sigmoid=tf.train.Saver()

In [44]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training 
    # uncomment below two line while you are restoring model
    #load_path = saver_sigmoid.restore(sess, model_path)
    #print "Model restored from file: %s" % save_path
    for epoch in range(training_epoch):
        avg_cost=0
        for i in range(batch_size):
            train_idx=np.random.randint(X_Train.shape[0],size=100)
            batch_x_Train=X_Train[train_idx,:]
            batch_y_Train=y_Train[train_idx]
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x_Train,y:batch_y_Train})
            avg_cost+=c/batch_size
            
        if epoch%display_step==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished"
    
    Predictions=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(Predictions,"float"))
    test_idx=np.random.randint(X_Test.shape[0],size=100)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    val_idx=np.random.randint(X_Val.shape[0],size=100)
    batch_x_Val=X_Val[val_idx,:]
    batch_y_Val=y_Val[val_idx]
    print "Training Accuracy:" ,accuracy.eval({x:batch_x_Train,y:batch_y_Train})
    print "Testing Accuracy:" ,accuracy.eval({x:batch_x_Test,y:batch_y_Test})
    print "Validation Accuracy:" ,accuracy.eval({x:batch_x_Val,y:batch_y_Val})
    # comment below two while you are restore the model
    #save_path=saver_sigmoid.save(sess,model_path)
    #print "Model saved in file: %s" % save_path
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:X_Test, y:y_Test})
    y_true = np.argmax(y_Test,1)
    print "Precision", sk.precision_score(y_true, y_pred)
    print "Recall", sk.recall_score(y_true, y_pred)
    print "confusion_matrix"
    print sk.confusion_matrix(y_true, y_pred)
    print "kappa score",cohen_kappa_score(y_true,y_pred)
    fpr, tpr, tresholds = sk.roc_curve(y_true, y_pred)
    sess.close()

Epoch: 0001 cost= 1.228664105
Epoch: 0002 cost= 0.678387476
Epoch: 0003 cost= 0.509715594
Epoch: 0004 cost= 0.457714663
Epoch: 0005 cost= 0.419929296
Epoch: 0006 cost= 0.384350294
Epoch: 0007 cost= 0.354540801
Epoch: 0008 cost= 0.340923613
Epoch: 0009 cost= 0.322776522
Epoch: 0010 cost= 0.305304616
Epoch: 0011 cost= 0.288729524
Epoch: 0012 cost= 0.269727160
Epoch: 0013 cost= 0.271325109
Epoch: 0014 cost= 0.244329674
Epoch: 0015 cost= 0.234174593
Optimization Finished
Training Accuracy: 0.94
Testing Accuracy: 0.73
Validation Accuracy: 0.67
Precision 0.0645161290323
Recall 0.017094017094
confusion_matrix
[[454  29]
 [115   2]]
kappa score -0.0595246854536


#### Model with tanh activation function

In [45]:
model_path = "/home/edward/Trained models/model_tanh.ckpt"
def multiplayer_perceptron_tanh(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.tanh(layer_1)
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.tanh(layer_2)
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [46]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron_tanh(x,weights,biases)

# define loss and optimizer
predictions=tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)
cost=tf.reduce_mean(predictions)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [47]:
# save model with sigmoid Activation function
#saver_tanh=tf.train.Saver()

In [48]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training 
    # uncomment below two line while you are restoring model
    #load_path = saver_sigmoid.restore(sess, model_path)
    #print "Model restored from file: %s" % save_path
    for epoch in range(training_epoch):
        avg_cost=0
        for i in range(batch_size):
            train_idx=np.random.randint(X_Train.shape[0],size=100)
            batch_x_Train=X_Train[train_idx,:]
            batch_y_Train=y_Train[train_idx]
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x_Train,y:batch_y_Train})
            avg_cost+=c/batch_size
            
        if epoch%display_step==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished"
    
    Predictions=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(Predictions,"float"))
    test_idx=np.random.randint(X_Test.shape[0],size=100)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    val_idx=np.random.randint(X_Val.shape[0],size=100)
    batch_x_Val=X_Val[val_idx,:]
    batch_y_Val=y_Val[val_idx]
    print "Training Accuracy:" ,accuracy.eval({x:batch_x_Train,y:batch_y_Train})
    print "Testing Accuracy:" ,accuracy.eval({x:batch_x_Test,y:batch_y_Test})
    print "Validation Accuracy:" ,accuracy.eval({x:batch_x_Val,y:batch_y_Val})
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:X_Test, y:y_Test})
    y_true = np.argmax(y_Test,1)
    print "Precision", sk.precision_score(y_true, y_pred)
    print "Recall", sk.recall_score(y_true, y_pred)
    print "confusion_matrix"
    print sk.confusion_matrix(y_true, y_pred)
    print "kappa score",cohen_kappa_score(y_true,y_pred)
    fpr, tpr, tresholds = sk.roc_curve(y_true, y_pred)
    # comment below two while you are restore the model
    #save_path=saver_tanh.save(sess,model_path)
    #print "Model saved in file: %s" % save_path
    sess.close()

Epoch: 0001 cost= 3.610109921
Epoch: 0002 cost= 2.239949865
Epoch: 0003 cost= 1.261066287
Epoch: 0004 cost= 0.860407704
Epoch: 0005 cost= 0.649729277
Epoch: 0006 cost= 0.416621132
Epoch: 0007 cost= 0.300595850
Epoch: 0008 cost= 0.275609066
Epoch: 0009 cost= 0.230118795
Epoch: 0010 cost= 0.219035891
Epoch: 0011 cost= 0.220502406
Epoch: 0012 cost= 0.194644471
Epoch: 0013 cost= 0.183065884
Epoch: 0014 cost= 0.176744824
Epoch: 0015 cost= 0.168487780
Optimization Finished
Training Accuracy: 0.95
Testing Accuracy: 0.74
Validation Accuracy: 0.81
Precision 0.0851063829787
Recall 0.034188034188
confusion_matrix
[[440  43]
 [113   4]]
kappa score -0.0709137090684


In [49]:
model_path = "/home/edward/Trained models/model_relu.ckpt"
def multiplayer_perceptron_relu(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.relu(layer_1)
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.relu(layer_2)
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [50]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron_relu(x,weights,biases)

# define loss and optimizer
predictions=tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)
cost=tf.reduce_mean(predictions)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [51]:
# save model with sigmoid Activation function
#saver_relu=tf.train.Saver()

In [52]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training 
    # uncomment below two line while you are restoring model
    #load_path = saver_sigmoid.restore(sess, model_path)
    #print "Model restored from file: %s" % save_path
    for epoch in range(training_epoch):
        avg_cost=0
        for i in range(batch_size):
            train_idx=np.random.randint(X_Train.shape[0],size=100)
            batch_x_Train=X_Train[train_idx,:]
            batch_y_Train=y_Train[train_idx]
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x_Train,y:batch_y_Train})
            avg_cost+=c/batch_size
            
        if epoch%display_step==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished"
    
    Predictions=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(Predictions,"float"))
    test_idx=np.random.randint(X_Test.shape[0],size=100)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    val_idx=np.random.randint(X_Val.shape[0],size=100)
    batch_x_Val=X_Val[val_idx,:]
    batch_y_Val=y_Val[val_idx]
    print "Training Accuracy:" ,accuracy.eval({x:batch_x_Train,y:batch_y_Train})
    print "Testing Accuracy:" ,accuracy.eval({x:batch_x_Test,y:batch_y_Test})
    print "Validation Accuracy:" ,accuracy.eval({x:batch_x_Val,y:batch_y_Val})
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:X_Test, y:y_Test})
    y_true = np.argmax(y_Test,1)
    print "Precision", sk.precision_score(y_true, y_pred)
    print "Recall", sk.recall_score(y_true, y_pred)
    print "confusion_matrix"
    print sk.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = sk.roc_curve(y_true, y_pred)
    print "kappa score",cohen_kappa_score(y_true,y_pred)
    # comment below two while you are restore the model
    #save_path=saver_relu.save(sess,model_path)
    #print "Model saved in file: %s" % save_path
    sess.close()

Epoch: 0001 cost= 59548.488125000
Epoch: 0002 cost= 8611.800339966
Epoch: 0003 cost= 2458.260316582
Epoch: 0004 cost= 2429.383820686
Epoch: 0005 cost= 1068.878244543
Epoch: 0006 cost= 1094.142381911
Epoch: 0007 cost= 2241.816273003
Epoch: 0008 cost= 1826.386764469
Epoch: 0009 cost= 1576.241149607
Epoch: 0010 cost= 887.070541590
Epoch: 0011 cost= 539.564180584
Epoch: 0012 cost= 694.889221716
Epoch: 0013 cost= 1150.702815208
Epoch: 0014 cost= 1830.847847433
Epoch: 0015 cost= 1874.240559444
Optimization Finished
Training Accuracy: 0.99
Testing Accuracy: 0.72
Validation Accuracy: 0.69
Precision 0.130434782609
Recall 0.0769230769231
confusion_matrix
[[423  60]
 [108   9]]
kappa score -0.0560060343202
